In [1]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib
import sys

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [2]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, Scan  # noqa: E402, F401
from mri_data import utils
from mri_data import file_manager as fm
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [3]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [4]:
#! Set these variables
work_dir_name = "pineal1"

In [5]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
drive_root = Path("/media/smbshare")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
ensemble_out_dir = work_dir / "ensemble_output"


Scan the ensemble_out folder produced during the monai training to collect the predicted labels into a DataSet struct. Create another DataSet object with the same subjects and sessions but rooted in the main data location, then collect the manual labels 

In [6]:
ensemble_out_dataset = scan_3Tpioneer_bids(ensemble_out_dir, label="flair.t1_ensemble.nii.gz")
dataset = DataSet.dataset_like(ensemble_out_dataset)
dataset.dataroot = dataroot
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels("pineal", suffix_list=["CH", "SRS"])
dataset = dataset_proc.dataset
dataset.sort()
ensemble_out_dataset.sort()

2024-10-24 16:45:48.725 | INFO     | monai_training.preprocess:prepare_labels:164 - Prepare Labels
  0%|          | 0/8 [00:00<?, ?it/s]2024-10-24 16:45:48.734 | INFO     | monai_training.preprocess:prepare_labels:223 - Found label pineal-CH.nii.gz for Scan(subid=1188, sesid=20200720)
2024-10-24 16:45:48.740 | INFO     | monai_training.preprocess:prepare_labels:223 - Found label pineal-SRS.nii.gz for Scan(subid=1029, sesid=20170816)
2024-10-24 16:45:48.745 | INFO     | monai_training.preprocess:prepare_labels:223 - Found label pineal-SRS.nii.gz for Scan(subid=1265, sesid=20180127)
2024-10-24 16:45:48.751 | INFO     | monai_training.preprocess:prepare_labels:223 - Found label pineal-CH.nii.gz for Scan(subid=1010, sesid=20180208)
2024-10-24 16:45:48.757 | INFO     | monai_training.preprocess:prepare_labels:223 - Found label pineal-SRS.nii.gz for Scan(subid=2187, sesid=20200731)
2024-10-24 16:45:48.762 | INFO     | monai_training.preprocess:prepare_labels:223 - Found label pineal-SRS.nii.

In [7]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    print("Scan: ", scan.subid, scan.label_path)
    print("Prediction: ", inference.subid, inference.label_path)

Scan:  1010 /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz
Prediction:  1010 /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz
Scan:  1029 /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz
Prediction:  1029 /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble.nii.gz
Scan:  1188 /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/pineal-CH.nii.gz
Prediction:  1188 /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble.nii.gz
Scan:  1265 /media/smbshare/3Tpioneer_bids/sub-ms1265/ses-20180127/pineal-SRS.nii.gz
Prediction:  1265 /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1265/ses-20180127/flair.t1_ensemble.nii.gz
Scan:  1272 /media/smbshare/3Tpioneer_bids/sub-ms1272/ses-20211105/pineal-SRS.nii.gz
Prediction:  1272 /home/s

Calculate Dice Scores

In [8]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    seg1 = nib.load(scan.label_path).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2)
    print("{}: {:0.2f}".format(scan.subid, score))

1010 1010
Dice Score: 0.5701754385964912
1029 1029
Dice Score: 0.5442477876106194
1188 1188
Dice Score: 0.4939271255060729
1265 1265
Dice Score: 0.6403785488958991
1272 1272
Dice Score: 0.6455026455026455
1280 1280
Dice Score: 0.09895833333333333
1321 1321
Dice Score: 0.3076923076923077
2187 2187
Dice Score: 0.8205128205128205


Save a new version of the predicted label that has a labelling index of 2 so that it appears as a different color in itksnap

In [12]:
for inference in ensemble_out_dataset:
    label_path = inference.label_path
    new_label_path = label_path.with_name(f"{fm.nifti_name(label_path)}_val2.nii.gz")
    utils.set_label_value(label_path, new_label_path, 2)

Produce commands to open images and labels in itksnap

In [15]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.root / new_label_path.name
    print("itksnap", "-g", flair, "-o", t1, "-s", prediction, manual)

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble_val2.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble_val2.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble_val2.nii.gz /media/smbshare/3Tpioneer_bids

Produce commands to open images and labels in freeview

In [23]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("freeview", flair, t1, f"{prediction}:colormap=heat", f"{manual}:colormap=heat")

freeview /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble.nii.gz:colormap=heat